# Creating a Composite Image
### ECOSTRESS Tutorials
###### This code is to be used when you have a collection of images that you would like to make a composite of. It works best if you have already cloud masked or QCed your images (See the cloud masking and QC tutorials). 
###### This code is written to make a composite of Land Surface Temperature (LST) files, but may be modified for use with other ECOSTRESS data products.

### Import Necessary Libraries

In [ ]:
import os
from os import makedirs
import numpy as np
import pandas as pd
import rioxarray
import rioxarray.merge
from glob import glob
from os.path import join, basename, splitext
from datetime import datetime
import hvplot.xarray


### Define Input and Output Locations, and List Files

In [ ]:
#Define input and output folders
#Replace with the path to the folder with the images you want to make composites of
input_directory = r"Replace_this_text_with_folder_path" 
# Replace with the path to the folder wheree you want the composite images to be saved
output_directory = r"Replace_this_text_with_folder_path" 
#This line makes sure that the output directory exists 
makedirs(output_directory, exist_ok=True)

#Set aesthetics 
alpha = 0.7 #Sets the transparency of the image to 70%
fig_width_px = 1080 #Defines the size of the figure
fig_height_px = 720

# Find all masked ST files in the output directory
ST_masked_filenames = sorted(glob(join(input_directory, "*_ST.tif")))

### Create a Median Composite of the Images

In [ ]:
#Opens the rasters using rioxarray
ST_masked_rasters = [rioxarray.open_rasterio(filename).squeeze("band", drop=True) for filename in ST_masked_filenames]
#Merges all the rasters into one array
ST_composite = rioxarray.merge.merge_arrays(ST_masked_rasters, nodata=np.nan)
#Calculates the median for each pixel
ST_composite.data = np.nanmedian(np.stack([raster.rio.reproject_match(ST_composite).data for raster in ST_masked_rasters]), axis=0)
#Replaces 0s with NaN
ST_composite.data = np.where(ST_composite.data == 0, np.nan, ST_composite.data)
#Create an output file path by combining the output folder loation with a file name
#Can replace the part in quotations with your own file name, make sure to wrap in quotes and add .tif
output_file_path = join(output_directory, "Replace_this_text_with_file_name.tif")
#Saves the Composite Raster
ST_composite.rio.to_raster(output_file_path)
#Reprojects the data and plots it
ST_composite.rio.reproject("EPSG:4326").hvplot.image(
    cmap="jet", #Uses the jet colorway
    tiles="OSM", #Uses Open Steet Maps as the background
    alpha=alpha,
    title = "ECOSTRESS Composite Image", #Can change title of map here
    width=fig_width_px,
    height=fig_height_px,
    clim=tuple(np.nanquantile(ST_composite.data, [0.02, 0.98]))
)